In [60]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [61]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
import torch
import numpy as np
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'

### Unexplored Params

In [62]:
params = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()

In [82]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

In [65]:
#identify explored param space

def get_unexplored_params(params, max_speed, max_spacing, min_speed, min_spacing, n = 196, seed = 1):
        """
        finds unexplored parameters
        inputs: tensor of [[], [], []], each size (1,14)
        outputs: list of n unnormalized tuples each of size (1,3)"""
        param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
        #get set of all possible parameter combinations
        param_all_comb = set(list(itertools.product(*param_space)))#[list(tup) for tup in itertools.product(*param_space)]
        wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
        #get set of all training parameter combinations
        train_param_space = set([tuple(x[:2] + [wattages[np.argmax(x[2:])]]) for x in params.tolist()]) #29988
        #find the combination parameters NOT in the training space
        unused_params = list(param_all_comb.symmetric_difference(train_param_space)) #len() = 5292
        random.seed = seed
        indices = random.sample(range(len(unused_params)), n)
        return [unused_params[i] for i in indices]

def unexplored_params_to_tensors(x):
        new_x = []
        wattage_idxs = {
                0.2: 0,
                0.3: 1,
                0.4: 2,
                0.5: 3,
                0.6: 4,
                0.7: 5,
                0.8: 6,
                0.9: 7,
                1.0: 8,
                1.1: 9,
                1.2: 10,
                1.3: 11,
        }      
        for i in x:
                i = list(i)
                holder = i[:2] + np.zeros(12).tolist()
                holder[wattage_idxs[i[2]] + 2] = 1.0
                new_x.append(holder)

        laser_params = torch.FloatTensor(new_x)
        return laser_params / laser_params.max(0).values

#[torch.cat((x[:2], torch.tensor(np.array([wattages[np.argmax(x[2:])]])))) for x in params]#[:5]
#[x[:2] for x in params]#[:5]

# train_param_space = denormalize_decode_result(params, max_speed, max_spacing, min_speed, min_spacing)
# train_param_space


In [66]:
ouput_unexplored = get_unexplored_params(params, max_speed, max_spacing, min_speed, min_spacing, n = 196, seed = 1)
norm_unexplored_laser_params = unexplored_params_to_tensors(ouput_unexplored)

# Inverse Model Only


In [67]:
from src.models import InverseModel

In [79]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-ann-inconel-2022-07-07_02-38.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-07_02-38.ckpt


In [81]:
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) #
with torch.no_grad():
    y_hat = i_model(x)
y_hat.shape

/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.layers.4.block.0.bias', 'direct_model.model.2.layers.5.block.0.weight', 'direct_model.model.2.layers.5.block.0.bias', 'direct_model.model.2.layers.6.block.0.weight', 'direct_model.model.2.layers.6.block.0.bias', 'direct_model.model.2.layers.7.block.0.weight', 'direct_model.model.2.layers.7.block.0.bias'

torch.Size([1, 14])

In [39]:
# y_final2 = onehot_to_continuous2(y_hat)
print(denormalize_decode_result(y_hat,max_speed, max_spacing, min_speed, min_spacing))

#if you want hardcoded max, min values instead
#print(denormalize_decode_result(y_hat,700.0, 42.0, 10.0, 1.0))

tensor([203.2477,  11.1895,   0.3000])


# Direct Model Only


In [75]:
from src.models import DirectModel

In [76]:
filepath = Path(
    f"{work_folder}/saved_best/best_D-ann-stainless-2022-07-07_03-08.ckpt")
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_D-ann-stainless-2022-07-07_03-08.ckpt


In [77]:
d_model = DirectModel.load_from_checkpoint(filepath)
d_model.eval()
# x = torch.randn(1, 14)
direct_y_hats = []
for i in norm_unexplored_laser_params:
    with torch.no_grad():
        direct_y_hats.append(d_model(i.reshape(1,14)))

In [71]:
direct_y_hats[0].shape

torch.Size([1, 800])

### Unexplored Direct --> Indirect

In [56]:
x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) 
x.shape

torch.Size([1, 800])

In [58]:
direct_y_hats[0].shape

torch.Size([1, 800])

In [83]:
model = InverseModel.load_from_checkpoint(filepath, strict=False)
model.eval()

x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) #
param_preds = []
for i in direct_y_hats:
    with torch.no_grad():
        param_preds.append(model(i))#model(i))


196

In [106]:
decoded_preds = [denormalize_decode_result(pred, max_speed, max_spacing, min_speed, min_spacing) for pred in param_preds]
for r in decoded_preds:
    r[0]=torch.round(r[0], decimals = -1)
    r[1]=torch.round(r[0], decimals = 0)
decoded_preds

[tensor([250.0000, 250.0000,   1.2000]),
 tensor([250., 250.,   1.]),
 tensor([40.0000, 40.0000,  0.7000]),
 tensor([420.0000, 420.0000,   0.9000]),
 tensor([230.0000, 230.0000,   1.2000]),
 tensor([300.0000, 300.0000,   1.2000]),
 tensor([130.0000, 130.0000,   1.1000]),
 tensor([140.0000, 140.0000,   1.1000]),
 tensor([110.0000, 110.0000,   1.1000]),
 tensor([170.0000, 170.0000,   1.1000]),
 tensor([260., 260.,   1.]),
 tensor([160.0000, 160.0000,   1.1000]),
 tensor([210.0000, 210.0000,   1.1000]),
 tensor([320., 320.,   1.]),
 tensor([110.0000, 110.0000,   1.1000]),
 tensor([160.0000, 160.0000,   1.1000]),
 tensor([170., 170.,   1.]),
 tensor([310.0000, 310.0000,   1.2000]),
 tensor([250.0000, 250.0000,   1.2000]),
 tensor([80.0000, 80.0000,  1.1000]),
 tensor([110.0000, 110.0000,   1.1000]),
 tensor([90.0000, 90.0000,  0.7000]),
 tensor([90.0000, 90.0000,  1.1000]),
 tensor([100.0000, 100.0000,   1.1000]),
 tensor([160., 160.,   1.]),
 tensor([130.0000, 130.0000,   1.1000]),
 tenso

In [107]:
space_params

{(23.1, 36.9, 0.7),
 (42.0, 17.2, 0.9),
 (43.3, 31.4, 0.7),
 (44.9, 33.8, 0.7),
 (45.9, 30.7, 0.7),
 (51.9, 41.9, 1.1),
 (55.4, 42.0, 1.1),
 (55.9, 42.0, 1.1),
 (56.6, 42.0, 1.1),
 (61.6, 42.0, 1.1),
 (64.5, 42.0, 1.1),
 (65.1, 42.0, 1.1),
 (66.0, 41.2, 0.7),
 (66.6, 40.9, 0.7),
 (69.5, 42.0, 1.1),
 (69.7, 36.4, 0.7),
 (71.6, 35.5, 0.7),
 (71.6, 42.0, 1.1),
 (75.4, 37.9, 0.7),
 (75.4, 42.0, 1.1),
 (77.0, 42.0, 1.1),
 (77.1, 41.9, 1.1),
 (77.7, 41.9, 1.1),
 (78.1, 42.0, 1.1),
 (78.9, 41.9, 1.1),
 (78.9, 42.0, 1.1),
 (80.1, 41.9, 1.1),
 (81.8, 39.1, 0.7),
 (84.1, 42.0, 1.1),
 (85.0, 41.7, 1.1),
 (85.6, 41.8, 1.1),
 (86.4, 41.6, 1.1),
 (87.7, 41.9, 1.1),
 (87.7, 42.0, 1.1),
 (88.0, 41.4, 1.1),
 (88.9, 41.7, 1.1),
 (88.9, 41.9, 1.1),
 (89.3, 40.0, 0.7),
 (89.3, 41.7, 1.1),
 (89.7, 38.9, 0.7),
 (89.9, 41.4, 1.1),
 (90.9, 41.8, 1.1),
 (91.0, 41.6, 1.1),
 (91.1, 40.5, 1.1),
 (91.4, 41.3, 1.1),
 (91.6, 41.7, 1.1),
 (92.5, 41.7, 1.1),
 (93.4, 41.7, 1.1),
 (94.0, 41.9, 1.1),
 (94.5, 41.5, 1.1),


In [108]:
param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
#get set of all possible parameter combinations
param_all_comb = set(list(itertools.product(*param_space)))
wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
space_params = set([tuple(np.around(x.tolist(), decimals= 1)) for x in decoded_preds]) #29988
        #find the combination parameters NOT in the training space
unused_params = list(param_all_comb.intersection(space_params)) #len() = 5292
#convert preds to set
len(unused_params)



3

In [109]:
#intersecting parameters!!!!!!!
unused_params

[(40.0, 40.0, 0.7), (20.0, 20.0, 0.7), (40.0, 40.0, 0.9)]

# ONNX

In [ ]:
# import onnxruntime 
# from pathlib import Path
# import numpy as np
# import os

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)

In [ ]:
# filepath = Path(f"{work_folder}/saved_best/best_inference_direct.onnx")

# ort_session = onnxruntime.InferenceSession(filepath)
# input_name = ort_session.get_inputs()[0].name
# ort_inputs = {input_name: np.random.randn(2, 14)}
# ort_outs = ort_session.run(None, ort_inputs)
